## Welcome to the (Un)Official KoboldCpp Colab Notebook
It's really easy to get started. Just press the two **Play** buttons below, and then connect to the **Cloudflare URL** shown at the end.

You can select a model from the dropdown or enter a HF model name into the Model feild, formatting should look like `KoboldAI/LLaMA2-13B-Tiefighter-GGUF`.

If a custom HF model name fails to load you may want to enable Manaul_Link and dircetly enter the link of the desired model. Formating for a manual URL should look like `https://huggingface.co/KoboldAI/LLaMA2-13B-Tiefighter-GGUF/resolve/main/LLaMA2-13B-Tiefighter.Q4_K_M.gguf`

For **13B** models The following congigurations are tested and known to work
- `Q4_K_M / 4096 context` Standard configuration
- `Q4_K_S / 6144 context` Increased context with a small cost to quality and speed
- `Q5_K_M / 4096 context` Increased quality at a small cost to speed

For **7B** models The following congigurations are tested and known to work
- `Q5_K_M / 4096 context` Standard configuration
- `Q5_K_M / 8192 context` Greatly increased context with a small cost to quality and speed

In [ ]:
#@title <-- Tap this if you play on Mobile { display-mode: "form" }
%%html
<b>Press play on the music player to keep the tab alive, then start KoboldCpp below</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
#@title <b>v-- Enter your model below and then click this to start Koboldcpp</b>

Model = "KoboldAI/LLaMA2-13B-Tiefighter-GGUF" #@param ["KoboldAI/LLaMA2-13B-Tiefighter-GGUF","TheBloke/ReMM-SLERP-L2-13B-GGUF","TheBloke/MythoMax-L2-13B-GGUF","TheBloke/Xwin-LM-13B-v0.2-GGUF","TheBloke/Stheno-L2-13B-GGUF","TheBloke/OpenHermes-2.5-Mistral-7B-GGUF","TheBloke/Mistral-7B-Claude-Chat-GGUF"]{allow-input: true}
Quant = "Q4_K_M" #@param ["Q4_K_S","Q4_K_M","Q5_K_M"] {allow-input: true}
ContextSize = 4096 #@param [2048,4096,6144,8192] {allow-input: true}
Manual_Link = False #@param {type:"boolean"}
Model_URL = "OPTIONAL: Enable Manual_Link and paste full model download link here" #@param [""]{allow-input: true}

LocalTunnelBackup = False #@param {type:"boolean"}


# Split the Model string into MODELPATH and MODELNAME
parts = Model.split('/')
MODELPATH = parts[0]
MODELNAME = parts[1].replace('-GGUF', '')

print("MODELPATH:", MODELPATH)
print("MODELNAME:", MODELNAME)

MODELURL = "https://huggingface.co/{MODELPATH}/{MODELNAME}-GGUF/resolve/main/{MODELNAME}.{Quant}.gguf"

if Manual_Link:
  MODELURL = Model_URL


%cd /content
!git clone https://github.com/LostRuins/koboldcpp
%cd /content/koboldcpp
kvers = !(cat koboldcpp.py | grep 'KcppVersion = ' | cut -d '"' -f2)
kvers = kvers[0]
!echo Finding prebuilt binary for {kvers}
!wget -c https://huggingface.co/concedo/koboldcpp/resolve/main/prebuilt_binaries/{kvers}.so
!test -f {kvers}.so && mv {kvers}.so koboldcpp_cublas.so || echo Prebuilt Binary Does Not Exist
!test -f koboldcpp_cublas.so && echo Prebuilt Binary Exists || make koboldcpp_cublas LLAMA_CUBLAS=1
!cp koboldcpp_cublas.so koboldcpp_cublas.dat
!rm model.ggml
!wget https://huggingface.co/{MODELPATH}/{MODELNAME}-GGUF/resolve/main/{MODELNAME}.{Quant}.gguf -nc -O model.ggml
MODELNAME = MODELNAME.lower()
import os

file_path = 'model.ggml'
file_size = os.path.getsize(file_path)  # Get the file size in bytes

if file_size < 1024 * 1024:  # 1MB = 1024*1024 bytes
    !rm {file_path}
else:
    print(f"The file {file_path} is larger than 1MB and will not be deleted.")
MODELURL = "https://huggingface.co/{MODELPATH}/{MODELNAME}-GGUF/resolve/main/{MODELNAME}.{Quant}.gguf"
!wget https://huggingface.co/{MODELPATH}/{MODELNAME}-GGUF/resolve/main/{MODELNAME}.{Quant}.gguf -nc -O model.ggml
if LocalTunnelBackup:
  !npm install -g localtunnel
  !nohup lt --port 5001 &
  !sleep 8
  !cat nohup.out
  FLAG = ""
else:
  FLAG = "--remotetunnel"


!python koboldcpp.py model.ggml --usecublas 0 --multiuser --gpulayers 99 --contextsize $ContextSize --hordeconfig concedo 1 1 {FLAG} --onready "curl -s ipecho.net/plain | xargs -I % echo If using the Localtunnel backup scroll up for your Localtunnel link, then use the following IP to verify: %"